In [ ]:
from simsopt.geo import SurfaceRZFourier
import numpy as np

# File for the desired boundary magnetic surface:
filename = "tests/test_files/input.LandremanPaul2021_QA" # "tests/test_files/input.LandremanPaul2021_QH_reactorScale_lowres"

# Initialize the boundary magnetic surface:
s = SurfaceRZFourier.from_vmec_input(filename, range="half period")

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt

def plot_modes(s:SurfaceRZFourier):
  extent = (-s.rc.shape[1]/2,s.rc.shape[1]/2,s.rc.shape[0]+0.5,-0.5)

  plt.subplot(211)
  plt.imshow(s.rc*1e3, norm="symlog", extent=extent)
  plt.colorbar()
  plt.title("Rcos")
  plt.ylabel("m")
  plt.subplot(212, sharex=plt.gca())
  plt.imshow(s.zs*1e3, norm="symlog", extent=extent)
  plt.colorbar()
  plt.title("Zsin")
  plt.ylabel("m")
  plt.xlabel("n")


## Vary B.n

In [ ]:
import stage_two_optimization

from simsopt.field import BiotSavart, Current, coils_via_symmetries
from simsopt.geo import create_equally_spaced_curves

ncoils = 4
R0 = 1.0 # Major radius for the initial circular coils
R1 = 0.5 # Minor radius for the initial circular coils
order = 5 # Number of Fourier modes describing each Cartesian component of each coil

# Create the initial coils:
base_curves = create_equally_spaced_curves(ncoils, s.nfp, stellsym=True, R0=R0, R1=R1, order=order)
base_currents = [Current(1e5) for i in range(ncoils)]
# Since the target field is zero, one possible solution is just to set all
# currents to 0. To avoid the minimizer finding that solution, we fix one
# of the currents:
base_currents[0].fix_all()

coils = coils_via_symmetries(base_curves, base_currents, s.nfp, True)
bs = BiotSavart(coils)
bs.set_points(s.gamma().reshape((-1, 3)))


In [ ]:
B2d = bs.B().reshape(s.unitnormal().shape)
Bn = np.sum(B2d * s.unitnormal(), axis=2)
plt.imshow(Bn)

## Vary the geometry 

In [ ]:
fourier_limits = []
mmax = max(s.m)
mmax = 3
nmax = max(s.n)
for m in reversed(range(0, mmax+1)):
  for n in reversed(range(0, nmax+1)):
    if m <= 1 and n<=0:
      continue
    for nsign in [-1, 1]:
      s.set_rc(m,n*nsign,0.0)
      s.set_zs(m,n*nsign,0.0)

      fourier_limits.append((m,n*nsign))
      
      plot_modes(s)
      s.plot()
      plt.show()

In [ ]:
from examples.Simple import stage_two_optimization_minimal
coilss = []

s = SurfaceRZFourier.from_vmec_input(filename, range="half period")

for m,n in fourier_limits:
  s.set_rc(m,n,0.0)
  s.set_zs(m,n,0.0)

  #plt.matshow(s.rc*1e3, norm="symlog")
  #plt.colorbar()
  #plt.title(f"Rcos {limit}")
  #plt.show()

  coils = stage_two_optimization_minimal.optimize(s)
  coilss.append(coils)

In [ ]:
from simsopt.geo import plot, LpCurveTorsion, LpCurveCurvature, fix_matplotlib_3d

%matplotlib widget
# Power of the norm, e.g. L2
p = 2

plt.close()
plt.gca().set_prop_cycle(color=plt.cm.plasma(np.linspace(0, 1, len(fourier_limits))))

for coils, limit in zip(coilss, np.linspace(0, 1, len(fourier_limits))): #[ff[0]*12+ff[1] for ff in fourier_limits]):
  curvatures = np.array([(LpCurveTorsion(coil.curve, p) + LpCurveCurvature(coil.curve, p)).J() for coil in coils])
  plt.scatter(np.array([limit]*len(curvatures)), curvatures)
plt.ylabel("L2 norm of Curvature")
plt.xlabel("Cutoff for |m|,|n| modes of B.n")
plt.subplots_adjust(right=0.7)
plt.legend([f"$m={m}, n={n}$" for m,n in fourier_limits],loc=(1.04, 0))
plt.show()

In [ ]:
%matplotlib inline
for coil_i in range(len(coilss[0])):
  ax = plt.figure(figsize=(16,10)).add_subplot(projection='3d')
  plt.gca().set_prop_cycle(color=plt.cm.plasma(np.linspace(0, 1, len(fourier_limits))))
  for coils in coilss:
    coils[coil_i].plot(ax=ax, show=False)
  plt.legend([f"$m={m},n={n}$" for m,n in fourier_limits])
  ax.set_prop_cycle(None) 
  fix_matplotlib_3d(ax)
  pass